In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [21]:
import narsil
from narsil.segmentation.run import loadNet, segmentPosDirectory
from torchvision import transforms
import matplotlib.pyplot as plt
from skimage.io import imread
import torch
from narsil.liverun.utils import resizeOneImage, tensorizeOneImage
import numpy as np
from scipy.signal import find_peaks
from pathlib import Path
import glob
from skimage.morphology import remove_small_objects

In [22]:
path = Path("C:\\Users\\Praneeth\\Documents\\Elflab\\Code\\testdata\\analysisData\\")

In [23]:
mask_filenames = []
for i in range(9):
    filename = path/str(i)/ "channelSegmentation"/ "0.tiff"
    mask_filenames.append(filename)

In [24]:
mask_filenames

[WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/0/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/1/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/2/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/3/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/4/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/5/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/6/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/7/channelSegmentation/0.tiff'),
 WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/8/channelSegmentation/0.tiff')]

In [6]:
mask_img = imread(mask_filenames[6]).astype('bool')
mask_img = remove_small_objects(mask_img, min_size = 1000)

In [7]:
mask_filenames[6]

WindowsPath('C:/Users/Praneeth/Documents/Elflab/Code/testdata/analysisData/6/channelSegmentation/0.tiff')

In [8]:
plt.imshow(mask_img)

In [9]:
hist = np.sum(mask_img, axis = 0)

In [42]:
plt.plot(hist)

In [10]:
peaks, _ = find_peaks(hist, distance = 25)

In [11]:
peaks

array([  11,   43,   78,  115,  151,  187,  228,  291,  327,  364,  398,
        434,  468,  503,  540,  574,  609,  643,  680,  716,  751,  782,
        821,  853,  888,  923,  957,  998, 1060, 1098, 1132, 1167, 1202,
       1239, 1273, 1307, 1344, 1380, 1412, 1446, 1482, 1518, 1552, 1587,
       1623, 1662, 1695, 1727, 1770, 1830, 1867, 1902, 1936, 1971, 2009,
       2043, 2081, 2116, 2147, 2181, 2220, 2255, 2287, 2321, 2357, 2396,
       2431, 2464, 2498, 2537, 2599, 2637, 2674, 2708, 2743, 2779, 2813,
       2847, 2883, 2919, 2951, 2988, 3022, 3061, 3092, 3128, 3162, 3197,
       3234, 3273, 3308, 3374, 3409, 3445, 3480, 3515, 3550, 3587, 3621,
       3658, 3692, 3727, 3764, 3798, 3834, 3869, 3906, 3940, 3975, 4011,
       4047, 4082], dtype=int64)

In [12]:
indices_with_larger_gaps = np.where(np.ediff1d(peaks) > 48)[0]

In [13]:
locations_before_barcode = peaks[indices_with_larger_gaps]
locations_after_barcode = peaks[indices_with_larger_gaps + 1]

# take the mean to obtain approximate center of the barcode region

locations_barcode = np.rint(np.mean((locations_before_barcode,
                                    locations_after_barcode), axis=0)).astype('int')

In [31]:
plt.figure(figsize=(16,5))
plt.plot(hist)
plt.plot(peaks, hist[peaks],'r*')
plt.plot(locations_before_barcode, hist[locations_before_barcode], 'k*')
plt.plot(locations_after_barcode, hist[locations_after_barcode], 'g*')
plt.show()

In [14]:
locations_barcode

array([ 260, 1029, 1800, 2568, 3341])

In [15]:
# count the number of peaks before and after the locations
peaks

array([  11,   43,   78,  115,  151,  187,  228,  291,  327,  364,  398,
        434,  468,  503,  540,  574,  609,  643,  680,  716,  751,  782,
        821,  853,  888,  923,  957,  998, 1060, 1098, 1132, 1167, 1202,
       1239, 1273, 1307, 1344, 1380, 1412, 1446, 1482, 1518, 1552, 1587,
       1623, 1662, 1695, 1727, 1770, 1830, 1867, 1902, 1936, 1971, 2009,
       2043, 2081, 2116, 2147, 2181, 2220, 2255, 2287, 2321, 2357, 2396,
       2431, 2464, 2498, 2537, 2599, 2637, 2674, 2708, 2743, 2779, 2813,
       2847, 2883, 2919, 2951, 2988, 3022, 3061, 3092, 3128, 3162, 3197,
       3234, 3273, 3308, 3374, 3409, 3445, 3480, 3515, 3550, 3587, 3621,
       3658, 3692, 3727, 3764, 3798, 3834, 3869, 3906, 3940, 3975, 4011,
       4047, 4082], dtype=int64)

### Mark 21 channels before the barcode if they are there before reaching the previous barcode, and accumulate the peaks

In [16]:
peaks

array([  11,   43,   78,  115,  151,  187,  228,  291,  327,  364,  398,
        434,  468,  503,  540,  574,  609,  643,  680,  716,  751,  782,
        821,  853,  888,  923,  957,  998, 1060, 1098, 1132, 1167, 1202,
       1239, 1273, 1307, 1344, 1380, 1412, 1446, 1482, 1518, 1552, 1587,
       1623, 1662, 1695, 1727, 1770, 1830, 1867, 1902, 1936, 1971, 2009,
       2043, 2081, 2116, 2147, 2181, 2220, 2255, 2287, 2321, 2357, 2396,
       2431, 2464, 2498, 2537, 2599, 2637, 2674, 2708, 2743, 2779, 2813,
       2847, 2883, 2919, 2951, 2988, 3022, 3061, 3092, 3128, 3162, 3197,
       3234, 3273, 3308, 3374, 3409, 3445, 3480, 3515, 3550, 3587, 3621,
       3658, 3692, 3727, 3764, 3798, 3834, 3869, 3906, 3940, 3975, 4011,
       4047, 4082], dtype=int64)

In [17]:
locations_barcode

array([ 260, 1029, 1800, 2568, 3341])

In [18]:
num_barcodes = len(locations_barcode)
channels = []

# if 6 barcode regions are detected, grab the 5 blocks in between
if num_barcodes ==6:
    for i in range(1, num_barcodes):
        print(i)
        indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                         peaks < locations_barcode[i]))[0]
        print(len(indices))
        
        channels.extend(list(peaks[indices]))
    
# if 5 are detected, check what you have before and after the first and last barcode
# respectively
elif num_barcodes == 5:
    indices_before = np.where(peaks < locations_barcode[0])[0]
    print(f"Before: {len(indices_before)}")
    indices_after = np.where(peaks > locations_barcode[-1])[0]
    print(f"After: {len(indices_after)}")
    
    # one of these is going to be 21 use that to cut out channels
    if len(indices_before) == 21:
        # do something
        for i in range(0, num_barcodes):
            print(i)
            if (i == 0):
                indices = np.where(peaks < locations_barcode[i])[0]
            else:
                indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                                  peaks < locations_barcode[i]))[0]
            print(len(indices))
            channels.extend(list(peaks[indices]))
        
    elif len(indices_after) == 21:
        # do something
        for i  in range(0, num_barcodes):
            print(i)
            if(i == num_barcodes - 1):
                print("adding after block")
                indices = np.where(peaks > locations_barcode[i])[0]
            else:
                indices = np.where(np.logical_and(peaks > locations_barcode[i],
                                                 peaks < locations_barcode[i+1]))[0]
            print(len(indices))
            channels.extend(list(peaks[indices]))
            
    else:
        # if neither of these is 21, it is a bit hard to grab the end channels.
        # safe to grab 4 blocks instead of 5.
        for i in range(0, num_barcodes):
            print(i)
            indices = np.where(np.logical_and(peaks > lcoations_barcode[i],
                                              peaks < locations_barcode[i+1]))[0]
            print(len(indices))
            channels.extend(list(peaks[indices]))
else:
    
    print("Barcode detection failed .. bailing out for this position")

Before: 7
After: 21
0
21
1
21
2
21
3
21
4
adding after block
21


In [19]:
len(channels)

105

In [28]:
locations_barcode

array([ 112,  892, 1658, 2424, 3200, 3991])

In [29]:
peaks

array([  10,   42,   76,  149,  187,  222,  254,  292,  327,  365,  396,
        433,  469,  505,  541,  575,  612,  648,  682,  716,  751,  787,
        821,  858,  926,  959,  994, 1029, 1062, 1095, 1133, 1167, 1202,
       1235, 1271, 1305, 1342, 1374, 1410, 1443, 1479, 1515, 1548, 1583,
       1627, 1689, 1723, 1758, 1794, 1827, 1863, 1899, 1934, 1970, 2003,
       2038, 2074, 2107, 2143, 2178, 2212, 2247, 2283, 2319, 2351, 2390,
       2459, 2495, 2529, 2567, 2599, 2635, 2673, 2703, 2742, 2779, 2809,
       2846, 2883, 2918, 2950, 2988, 3022, 3059, 3093, 3136, 3165, 3236,
       3270, 3307, 3341, 3377, 3416, 3451, 3487, 3521, 3557, 3591, 3628,
       3662, 3700, 3740, 3773, 3807, 3845, 3879, 3917, 3955, 4027, 4062],
      dtype=int64)

In [35]:
ans = np.where(np.logical_and(peaks < locations_barcode[1], peaks > locations_barcode[0]))[0]

In [36]:
peaks[ans]

array([149, 187, 222, 254, 292, 327, 365, 396, 433, 469, 505, 541, 575,
       612, 648, 682, 716, 751, 787, 821, 858], dtype=int64)

In [27]:
for i, location in enumerate(locations_barcode, 0):
    print(np.sum(peaks < locations_barcode[i]))
    print(np.sum(peaks > locations_barcode[i]))
    print("----")

9
103
----
30
82
----
51
61
----
72
40
----
93
19
----


In [28]:
peaks

array([  13,   50,   85,  119,  158,  191,  229,  261,  302,  366,  402,
        438,  476,  511,  547,  579,  619,  655,  689,  721,  757,  795,
        831,  863,  898,  932,  966, 1000, 1034, 1070, 1136, 1174, 1209,
       1247, 1283, 1314, 1349, 1383, 1418, 1457, 1493, 1527, 1561, 1597,
       1631, 1663, 1697, 1733, 1772, 1803, 1841, 1906, 1942, 1977, 2014,
       2049, 2085, 2121, 2155, 2188, 2220, 2257, 2294, 2328, 2365, 2399,
       2434, 2468, 2503, 2538, 2572, 2611, 2676, 2712, 2747, 2783, 2817,
       2851, 2887, 2923, 2957, 2993, 3028, 3063, 3097, 3131, 3168, 3203,
       3238, 3273, 3308, 3346, 3382, 3449, 3484, 3520, 3555, 3590, 3625,
       3660, 3697, 3732, 3767, 3804, 3837, 3873, 3909, 3944, 3981, 4015,
       4051, 4085], dtype=int64)

In [20]:
plt.imshow(mask_img)

In [10]:
plt.imshow(mask_img)

### Generating histograms to cut channels and blocks

In [65]:
def getAllBarcodes(mask_img, phase_image, minPeaksDistance=25, barcodeWidth=48,
                   minChannelLength=100, rowThreshold=10000,
                   plot=False):
    hist = np.sum(mask_img, axis = 0) > minChannelLength
    peaks, _ = find_peaks(hist, distance = minPeaksDistance)
    indices_with_larger_gaps = np.where(np.ediff1d(peaks) > barcodeWidth)[0]
    
    locations_before_barcode = peaks[indices_with_larger_gaps]
    locations_after_barcode = peaks[indices_with_larger_gaps + 1]
    
    # take the mean to obtain approximate center of the barcode region
    
    locations_barcode = np.rint(np.mean((locations_before_barcode,
                                        locations_after_barcode), axis=0)).astype('int')
    
    # now grab all the barcode images
    # we get the column numbers from the locations_barcode
    # but we need to find the row numbers, somehow. 
    # We use the same histogram and sum and threshold 
    sum_rows = np.sum(mask_img, axis = 1).astype('int')
    row_locations = np.argwhere(np.diff(np.sign(sum_rows - rowThreshold))).flatten()
    if len(row_locations) !=2:
        row_x1 = 400
        row_x2 = 1200
    else:
        row_x1 = row_locations[0]
        row_x2 = row_locations[1]
    # on 40x the channels size will be approximately.
    
    barcode_images = []
    for location in locations_barcode:
        barcode_img = phase_img[row_x1:row_x2, location-barcodeWidth//2 : location+barcodeWidth//2]
        barcode_images.append(barcode_img)
        
    if plot:
        plt.figure(figsize=(16,5))
        plt.plot(hist)
        plt.plot(peaks, hist[peaks],'r*')
        plt.plot(locations_before_barcode, hist[locations_before_barcode], 'k*')
        plt.plot(locations_after_barcode, hist[locations_after_barcode], 'g*')
        plt.show()
    return locations_barcode, barcode_images

def getAllChannels():
    pass

In [66]:
locations_barcode, barcode_images = getAllBarcodes(mask_img, phase_img, plot=True)

In [69]:
plt.imshow(barcode_images[2])